In [136]:
import boto3
import roc._manual_labels_config
import roc.state
from pathlib import Path 
config = roc._manual_labels_config.init()
state  = roc.state.init()

In [137]:
client = boto3.client('s3')
s3=boto3.resource('s3')
bucket_name = config['nn_info']['bucket']
bucket = s3.Bucket(bucket_name)

In [185]:
content = client.list_objects_v2(Bucket=bucket_name, Prefix = 'ROC/', Delimiter ='/')

try:
    prefix = [Path(item['Prefix']) for item in content['CommonPrefixes']]
except Exception as e:
    print(e)

In [187]:
prefix

[WindowsPath('ROC/LABEL_DATA'),
 WindowsPath('ROC/PROCESSED_DATA'),
 WindowsPath('ROC/SOLAR_DATA'),
 WindowsPath('ROC/TAG_DATA')]

In [204]:
def _listdir(bucket:str, prefix:str=''):
    prefix_path = prefix +'/' if prefix != '' else prefix
    file_structure = client.list_objects_v2(Bucket=bucket, Prefix=prefix_path, Delimiter='/')
    try:
        sub_folder = {Path(item['Prefix']).as_posix(): Path(item['Prefix']).relative_to(prefix).as_posix() for item in file_structure['CommonPrefixes']}
        return sub_folder 
    except KeyError as e:
        return None 

In [206]:
_listdir(bucket_name, 'ROC/LABEL_DATA')

In [207]:
x={}

In [210]:
len(list(x.keys()))

0

In [231]:
from aau.advancedanalytics_util import S3
from pathlib import Path 

class S3Manager(S3):
    def _listdir(self, bucket:str, prefix:str=''):
        prefix_path = prefix +'/' if prefix != '' else prefix
        file_structure = self.client.list_objects_v2(Bucket=bucket, Prefix=prefix_path, Delimiter='/')
        try:
            sub_folder_prefix = [Path(item['Prefix']).as_posix() for item in file_structure['CommonPrefixes']]
            sub_folder_rel_path = {prefix: [Path(item['Prefix']).relative_to(prefix).as_posix() for item in file_structure['CommonPrefix']]}
            return sub_folder_rel_path, sub_folder_prefix
        except KeyError as e:
            return {}, []

    def listdir(self, bucket:str):
        ls = self._listdir(bucket)
        next = list(ls.keys())
        status=True 

        while status:
            if len(next)==0:
                break
            prefix = next.pop()
            current_ls = self._listdir(bucket, prefix)
            next.extend(list(current_ls.keys()))
            ls.update(current_ls)
        return ls 

In [232]:
s3 = S3Manager(config['procdata_info'])

In [233]:
s3.listdir(bucket_name)

{'': ['ROC'],
 'ROC/LABEL_DATA': 'LABEL_DATA',
 'ROC/PROCESSED_DATA': 'PROCESSED_DATA',
 'ROC/SOLAR_DATA': 'SOLAR_DATA',
 'ROC/TAG_DATA': 'TAG_DATA'}